# Using the API to get custom data fields from the Civil War Maps collection

In [2]:
import requests

The function below takes as input the API query. For each result, it gets selected relevant fields. 

In [5]:
def get_items(url, items=[]):
    call = requests.get(url)
    data = call.json()
    results = data['results']
    for result in results:
        # don't get image from the collection-level result
        if "collection" not in result.get("original_format") and "web page" not in result.get("original_format"):
            item_data = {}
            item_data["item_url"] = result.get("id")
            item_data["title"] = result.get("title")
            item_data["shelf_id"] = result.get("shelf_id")
            item_data["description"] = result.get("description")[0]
            items.append(item_data)
    if data["pagination"]["next"] is not None: # make sure we haven't hit the end of the pages
        next_url = data["pagination"]["next"]
        get_items(next_url, items)
        
    return items


Query within the Civil War Maps collection on the term photocopy. (TODO: broader query on maps, targeting a data range)

In [6]:
item_list = get_items("https://www.loc.gov/collections/civil-war-maps/?fo=json&at=results,pagination&q=photocopy")

In [10]:
len(item_list)

418

Turn items into a CSV file:

In [16]:
import csv

with open("civil-war-maps-photocopies.csv", 'w') as output: 
    fieldnames = ["item_url", "title", "shelf_id", "description"]
    writer = csv.DictWriter(output, fieldnames=fieldnames)
    writer.writeheader()
    for item in item_list:
        writer.writerow(item)

How many lines are in the resulting CSV?

In [19]:
!wc -l civil-war-maps-photocopies.csv

     419 civil-war-maps-photocopies.csv


Using csvjoin (part of the [csvkit](https://csvkit.readthedocs.io/) toolkit) to stitch together the API data with the colors data from the [Library of Congress Colors](https://loc-colors.glitch.me/) project. The item's url is the key in each file and used to match rows between the files. 

In [14]:
!csvjoin -c item_url,url civil-war-maps-photocopies.csv civil-war-maps.csv > civil-war-maps-joined.csv

In [18]:
!head -2 civil-war-maps-joined.csv

item_url,title,shelf_id,description,color1,color2,color3,color4,color5,color6
http://www.loc.gov/item/2006626053/,"Small sketch, first Manassas, July 21, 1861 : situation on Henry Hill at 3 P.M.",G3884.M25S5 1861 .R8,"['Hand colored positive photocopy. LC Civil War maps (2nd ed.), 570 Description derived from published bibliography. Available also through the Library of Congress Web site as a raster image.']",#392a26,#97836e,#b8a48f,#cdc5b4,#625f54,#f0ebd5


This CSV file can now be opened in one of many applications, including Google Sheets or Excel. 

Using Google App Scripts let me make the background and font color match the hexadecimal color value in the cell. This allows someone to browse the colors and metadata side by side, especially for when the color conveys information about format (e.g. cyanotype). 

```
function colorize() {
  var r = SpreadsheetApp.getActive().getSheetByName('civil-war-maps-joined').getRange('E2:J420');
  r.setBackgrounds(r.getValues());
  r.setFontColors(r.getValues());
}
```

